In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.2)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.93 sec - Loss 0.061286 - ACC 59.40% - ACC Mean 59.40% - AUC 63.53% - AUC Mean 63.53% - Deter 000
Ite 00050 - 1.32 sec - Loss 0.012290 - ACC 59.50% - ACC Mean 59.23% - AUC 62.83% - AUC Mean 63.09% - Deter 032
Ite 00100 - 1.33 sec - Loss 0.006530 - ACC 57.65% - ACC Mean 59.78% - AUC 62.49% - AUC Mean 63.60% - Deter 037
Ite 00113 - 1.80 sec - Loss 0.005813 - ACC 59.45% - ACC Mean 59.82% - AUC 62.42% - AUC Mean 63.70% - Deter 050
Early stop ite 113, rollback to correction of ite 63, whith acc of 62.3% and auc of 66.0%
OUR METHOD RUN 1/10 - acc: 62.3% +- 0.0% - auc: 66.0% +- 0.0%
Ite 00000 - 1.37 sec - Loss 0.135436 - ACC 60.25% - ACC Mean 60.25% - AUC 63.69% - AUC Mean 63.69% - Deter 000
Ite 00050 - 1.47 sec - Loss 0.051123 - ACC 60.25% - ACC Mean 59.99% - AUC 63.98% - AUC Mean 63.72% - Deter 038
Ite 00100 - 1.54 sec - Loss 0.032107 - ACC 60.25% - ACC Mean 60.50% - AUC 65.10% - AUC Mean 64.54% - Deter 038
Ite 00112 - 1.79 sec - Loss 0.029435 - ACC 60.10% - ACC Mean 60.47% - A

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 59.6% +- 0.0% - auc: 63.03% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 58.95% +- 1.18% - auc: 63.58% +- 0.58%
SFIL RUN 2/10 - acc: 59.5% +- 2.52% - auc: 63.34% +- 2.89%
SFIL RUN 3/10 - acc: 59.35% +- 2.81% - auc: 62.78% +- 4.41%
SFIL RUN 4/10 - acc: 59.25% +- 1.38% - auc: 63.48% +- 1.25%
SFIL RUN 5/10 - acc: 58.75% +- 3.56% - auc: 62.76% +- 3.49%
SFIL RUN 6/10 - acc: 59.2% +- 1.12% - auc: 62.03% +- 2.57%
SFIL RUN 7/10 - acc: 57.5% +- 1.57% - auc: 62.11% +- 1.48%
SFIL RUN 8/10 - acc: 58.15% +- 1.09% - auc: 61.92% +- 1.05%
SFIL RUN 9/10 - acc: 59.85% +- 1.99% - auc: 63.89% +- 2.38%
SFIL RUN 10/10 - acc: 60.35% +- 0.56% - auc: 63.9% +- 1.07%
SFIL GLOBAL - acc: 59.09% +- 0.77% - auc: 62.98% +- 0.73%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 59.2% +- 1.21% - auc: 61.71% +- 1.39%
PFIL RUN 2/10 - acc: 57.85% +- 1.78% - auc: 61.52% +- 1.75%
PFIL RUN 3/10 - acc: 57.65% +- 2.22% - auc: 61.49% +- 2.38%
PFIL RUN 4/10 - acc: 56.55% +- 2.25% - auc: 60.08% +- 3.36%
PFIL RUN 5/10 - acc: 59.1% +- 1.78% - auc: 63.77% +- 1.53%
PFIL RUN 6/10 - acc: 57.35% +- 1.95% - auc: 60.13% +- 3.05%
PFIL RUN 7/10 - acc: 56.75% +- 1.82% - auc: 59.88% +- 1.8%
PFIL RUN 8/10 - acc: 58.95% +- 1.96% - auc: 62.61% +- 1.13%
PFIL RUN 9/10 - acc: 56.15% +- 1.27% - auc: 59.95% +- 1.66%
PFIL RUN 10/10 - acc: 58.25% +- 2.15% - auc: 61.0% +- 2.37%
PFIL GLOBAL - acc: 57.78% +- 1.04% - auc: 61.21% +- 1.22%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 58.45% +- 3.4% - auc: 62.2% +- 3.15%
SPOL RUN 2/10 - acc: 58.05% +- 1.46% - auc: 61.61% +- 2.34%
SPOL RUN 3/10 - acc: 59.4% +- 1.87% - auc: 63.38% +- 1.15%
SPOL RUN 4/10 - acc: 57.6% +- 2.12% - auc: 61.06% +- 2.81%
SPOL RUN 5/10 - acc: 58.45% +- 1.17% - auc: 61.97% +- 1.76%
SPOL RUN 6/10 - acc: 60.55% +- 2.96% - auc: 63.51% +- 1.72%
SPOL RUN 7/10 - acc: 59.4% +- 2.45% - auc: 61.6% +- 1.82%
SPOL RUN 8/10 - acc: 58.8% +- 3.05% - auc: 62.61% +- 3.84%
SPOL RUN 9/10 - acc: 57.95% +- 0.87% - auc: 61.42% +- 0.78%
SPOL RUN 10/10 - acc: 59.5% +- 2.85% - auc: 63.15% +- 2.24%
SPOL GLOBAL - acc: 58.82% +- 0.85% - auc: 62.25% +- 0.83%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 57.7% +- 3.08% - auc: 61.31% +- 3.29%
PPOL RUN 2/10 - acc: 58.35% +- 1.26% - auc: 62.94% +- 1.87%
PPOL RUN 3/10 - acc: 57.5% +- 1.54% - auc: 61.05% +- 1.87%
PPOL RUN 4/10 - acc: 58.8% +- 1.34% - auc: 61.77% +- 1.09%
PPOL RUN 5/10 - acc: 59.75% +- 4.34% - auc: 63.63% +- 4.1%
PPOL RUN 6/10 - acc: 58.15% +- 2.9% - auc: 61.29% +- 3.37%
PPOL RUN 7/10 - acc: 58.7% +- 2.42% - auc: 61.91% +- 3.02%
PPOL RUN 8/10 - acc: 58.3% +- 1.64% - auc: 62.76% +- 1.87%
PPOL RUN 9/10 - acc: 60.3% +- 2.56% - auc: 63.14% +- 3.39%
PPOL RUN 10/10 - acc: 58.3% +- 2.82% - auc: 60.96% +- 2.69%
PPOL GLOBAL - acc: 58.58% +- 0.82% - auc: 62.08% +- 0.92%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
